## Health Condition using 'hcond' and 'hcondnew'
### By Gavin Qu - May 23rd 2024
#### Data Extraction 

-	Encode hcond and new healthcond variables correctly 
-	Note that individuals are asked about pre-existing health conditions on their first interview in the UKHLS – the hcond (i) variable, where i codes different conditions – and then asked whether they have developed new conditions in subsequent interviews – the hcondn(i) variable in waves 1-9 and hcondnew(i) in waves 10 onwards.
-	hcond in wave 1 and new entrants for succeeding waves, hcondn in wave 1-9, hcondnew in wave 10-13

For example, hcond1-20 has 1, 3-13 waves, and it's for new interviewees only. While hcondn1-20 have 2-9 waves asking the existing interviewees about newly devloped conditions, and hcondnew1-20 have wave 9-13 for the same questions. 
hcond21 and hcondnew21 only exist from wave 10-13, while hcondnew22 only exist from 10-13. 